# Hybrid CNN + Quantum Neural Network for CIFAR-10
Implements a density QNN architecture based on the paper's framework

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pennylane as qml
import numpy as np

## Environment Setup
Load AWS configuration from .env file

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import clear_output

# Load environment variables from .env file
load_dotenv()

# Function to safely get and mask sensitive environment variables
def get_masked_env(var_name):
    value = os.getenv(var_name, '')
    if value and var_name in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY']:
        return value[:4] + '*' * (len(value) - 8) + value[-4:]
    return value

# Display current configuration
print("Current AWS Configuration:")
print(f"Region: {get_masked_env('AWS_DEFAULT_REGION')}")
print(f"Access Key ID: {get_masked_env('AWS_ACCESS_KEY_ID')}")
print(f"Secret Access Key: {get_masked_env('AWS_SECRET_ACCESS_KEY')}")
print(f"Braket Device: {get_masked_env('BRAKET_DEVICE')}")

# Function to update AWS configuration
def update_aws_config(region=None, access_key=None, secret_key=None, device_arn=None):
    if region:
        os.environ['AWS_DEFAULT_REGION'] = region
    if access_key:
        os.environ['AWS_ACCESS_KEY_ID'] = access_key
    if secret_key:
        os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key
    if device_arn:
        os.environ['BRAKET_DEVICE'] = device_arn
    
    clear_output()
    print("Updated AWS Configuration:")
    print(f"Region: {get_masked_env('AWS_DEFAULT_REGION')}")
    print(f"Access Key ID: {get_masked_env('AWS_ACCESS_KEY_ID')}")
    print(f"Secret Access Key: {get_masked_env('AWS_SECRET_ACCESS_KEY')}")
    print(f"Braket Device: {get_masked_env('BRAKET_DEVICE')}")

# Install python-dotenv if not already installed
try:
    import dotenv
except ImportError:
    !pip install python-dotenv

In [2]:
# Load CIFAR-10 sample
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
images, labels = next(iter(trainloader))

print(f"Loaded batch: {images.shape}")

Files already downloaded and verified
Loaded batch: torch.Size([4, 3, 32, 32])


In [ ]:
# Amazon Braket SV1 simulator device
dev = qml.device(
    "braket.aws.qubit",
    device_arn=os.getenv('BRAKET_DEVICE'),
    wires=4,
    shots=1000
)

NoRegionError: You must specify a region.

In [ ]:
# Density QNN: Sub-unitary quantum circuit (RBS-based)
@qml.qnode(dev, interface="torch")
def quantum_sub_circuit(inputs, weights):
    # Data encoding
    for i in range(4):
        qml.RY(inputs[i], wires=i)
    
    # Parameterized RBS-inspired gates
    for i in range(4):
        qml.RZ(weights[i], wires=i)
    
    # Entanglement (CNOT ladder)
    for i in range(3):
        qml.CNOT(wires=[i, i+1])
    
    # Second rotation layer
    for i in range(4):
        qml.RY(weights[i+4], wires=i)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [ ]:
# Hybrid CNN + Density Quantum Model
class HybridDensityQNN(nn.Module):
    def __init__(self, num_sub_unitaries=2):
        super(HybridDensityQNN, self).__init__()
        
        # CNN feature extractor
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 4)
        
        # Density QNN: K sub-unitaries with independent parameters
        self.K = num_sub_unitaries
        self.quantum_layers = nn.ModuleList([
            qml.qnn.TorchLayer(quantum_sub_circuit, {"weights": (8,)})
            for _ in range(self.K)
        ])
        
        # Trainable mixing coefficients α_k (ensure sum to 1 via softmax)
        self.alpha = nn.Parameter(torch.ones(self.K))
        
        # Final classifier
        self.fc2 = nn.Linear(4, 10)
    
    def forward(self, x):
        # CNN feature extraction
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.tanh(self.fc1(x))
        
        # Density QNN: weighted sum of sub-unitaries
        alpha_norm = torch.softmax(self.alpha, dim=0)
        quantum_out = sum(alpha_norm[k] * self.quantum_layers[k](x) for k in range(self.K))
        
        # Classification
        out = self.fc2(quantum_out)
        return out

In [ ]:
# Initialize and test forward pass
model = HybridDensityQNN(num_sub_unitaries=2)
output = model(images)

print(f"Input shape: {images.shape}")
print(f"Output shape: {output.shape}")
print(f"Mixing coefficients α: {torch.softmax(model.alpha, dim=0).detach()}")
print(f"\nOutput logits:\n{output}")